In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [7]:
df = pd.read_csv('estadisticas_fantasy.csv')
df

,Unnamed: 0,Nombre,Posición,Equipo,Puntos,Media Puntos Relevo,Puntos Relevo,Precio,Media,Partidos,...,Penaltis Fallados,Goles en Contra,Tarjetas Rojas,Paradas,Penaltis Cometidos,Tarjetas Amarillas,Segundas Amarillas,Penaltis Provocados,Penaltis Parados,Goles en Propia Puerta
0,0,Lewandowski,DL,FC Barcelona,146,2.85,37,127.371.980 €,11.23,13,...,1,8,0,0,0,1,0,0,0,0
1,1,Raphinha,DL,FC Barcelona,143,3.23,42,125.574.940 €,11.00,13,...,0,9,0,0,0,1,0,2,0,0
2,2,Lamine Yamal,DL,FC Barcelona,137,3.58,43,130.755.379 €,11.42,12,...,0,8,0,0,0,1,0,1,0,0
3,3,Vinícius Jr.,DL,Real Madrid,118,3.08,37,122.524.964 €,9.83,12,...,0,9,0,0,0,4,0,2,0,0
4,4,Mbappe,DL,Real Madrid,102,2.82,31,113.861.649 €,9.27,11,...,0,8,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
585,585,Vallejo,DF,Real Madrid,-1,0.00,0,636.541 €,-0.11,9,...,0,2,0,0,0,0,0,0,0,0
586,586,Adrián,PO,Real Betis,-2,0.00,0,573.596 €,-0.15,13,...,0,0,0,0,0,2,0,0,0,0
587,587,Rodrigo Abajas,DF,Valencia CF,-2,0.00,0,509.100 €,-0.33,6,...,0,2,0,0,0,1,0,0,0,0
588,588,Juan Carlos,PO,Girona,-3,0.00,0,576.338 €,-0.33,9,...,0,0,1,0,0,0,0,0,0,0


In [8]:
print("num_atributs: ", len(df.keys()))
nan_columns = df.columns[df.isna().sum()>0].tolist()
print(nan_columns)

num_atributs:  30
[]


In [9]:
corr_matrix = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5)
plt.title('Correlation Matrix')
plt.show()


ValueError: could not convert string to float: 'Lewandowski'